# Task 1
В тетрадке реализована биграмная языковая модель (при генерации учитывается информация только о 1 предыдущем слове). Реализуйте триграмную модель и сгенерируйте несколько текстов. Сравните их с текстами, сгенерированными биграмной моделью. 
Можно использовать те же тексты, что в семинаре, или взять какой-то другой (на английском или русском языке).  

Делать это задание будет легче после прочтения первых 7 страниц вот этой главы из Журафского - https://web.stanford.edu/~jurafsky/slp3/3.pdf

In [260]:
from nltk.tokenize import sent_tokenize
from razdel import tokenize as razdel_tokenize
from string import punctuation

class Preporcessor():
    """
        a class to preprocess a txt file availabe through the path
    """
    def __init__(self, path):
        self.path = path # a path to a txt file
        self.text = ""
        self.tokens_by_sentence = []
        
    def extractContent(self):
        """ loads the content of the file """
        with open(self.path, 'r', encoding="utf-8") as f: # open in readonly mode
            self.text = f.read()
            
    def tokenize_by_sentence(self, n=0):
        """
            tokenizes a text into a list of tokens by sentence
            inserts n times <start> and n times <end> of tags
            at the beginning and the end of a sentence
        """
        tmp = [self.sp_tags(self.normalize_text(sent), n) for sent in self.get_sentences()]
        self.tokens_by_sentence = tmp
    
    def get_sentences(self) -> list:
        """ uses nltk sent_tokenize to split the text into a list of sentences """
        return(sent_tokenize(self.text[0:250000])) # limit the coprus

    def normalize_text(self, text) -> list:
        """ normalizes text and turns it into a list of tokens """
        to_remove = punctuation + '«»“”'
        tmp = [word.text.strip(to_remove) for word in razdel_tokenize(text)]
        tmp = [word.lower() for word in tmp if word and len(word) < 20]
        return(tmp)
    
    def sp_tags(self, tokens, n=0) -> list:
        """ recieves a list of tokens and returns them with special tags """
        if n >= 0:
            return(['<start>'] * n + tokens + ['<end>'] *n)
        else:
            return(ValueError)

### Tokens

In [261]:
news = Preprocessor('data/lenta.txt')
news.extractContent()
news.tokenize_by_sentence(2) # add 2 special tokens
# news.tokenize_by_sentence(0) # add no special tokens

### Generator

In [267]:
from collections import Counter
import numpy as np

class TextGenerator():
    """
        a class to generate text based on a list tokens split by sentences
        input: [['tokens', 'of', 'a', 'single', 'sentence']]
        
        !!! modify to work on any ngram >= 2. Now it does not suit any to the full extent !!!
    """
    def __init__(self, tokens, ngram_length=2):
        if ngram_length < 2: # for bigrams and larger series only
            return(ValueError)
        self.tokens = tokens
        self.ngram_length = ngram_length
        self.ngrams = [] # [0] (n-1)grams [1] ngrams
        self.id2unigram = []
        self.unigram2id = {}
        self.id2ngram = []
        self.ngram2id = {}
        self.matrix = np.empty(0)
        
     
    def generate_ngrams(self):
        """ generate ngrams from sentences """
        self.generate_counters() # generate enough counters for ngrams
        
        for sentence in self.tokens:
            self.ngrams[0].update(sentence)
            self.ngrams[1].update(
                self.ngrammer(sentence, (self.ngram_length-1)) # gerenate ngrams of n-1 lenght
                ) 
            self.ngrams[2].update(
                self.ngrammer(sentence, self.ngram_length) # generate ngrams
                )

    def generate_counters(self):
        """
            generates a number of ngrams Counter() to store in self.ngrams
        """
        [self.ngrams.append(Counter()) for i in range(3)]
        
    def ngrammer(self, tokens, n=2) -> list:
        """ return ngrams """
        ngrams = []
        for i in range(0,len(tokens)-n+1):
            ngrams.append(' '.join(tokens[i:i+n]))
        return(ngrams)
    
    def build_matrix(self):
        self.matrix = np.zeros(
            (len(self.ngrams[1]), # rows with bigrams
             len(self.ngrams[0]) # columns with unigrams
            ))
        
        self.id2unigram = list(self.ngrams[0]) # index unigrams
        self.unigram2id = {unigram:i for i, unigram in enumerate(self.id2unigram)}
        self.id2ngram = list(self.ngrams[1]) # index (n-1)grams
        self.ngram2id = {ngram:i for i, ngram in enumerate(self.id2ngram)}
        
        
        for ngram in self.ngrams[-1]:
            tmp = ngram.split()
            prev = " ".join(tmp[:-1]) # save ngram without last token
            self.matrix[self.ngram2id[prev]][self.unigram2id[tmp[-1]]] = (self.ngrams[-1][ngram]/
                                                                            self.ngrams[-2][prev])
            
    def generate_text(self, n=100, start='<start> <start>') -> str:
        """ generates text with a calculated matrix """
        text = []
        current_idx = self.ngram2id[start]
    
        for i in range(n):

            chosen = np.random.choice(self.matrix.shape[1], p=self.matrix[current_idx])
            text.append(self.id2unigram[chosen])

            if self.id2unigram[chosen] != '<end>':
                current_idx = self.ngram2id[
                    self.id2ngram[current_idx].split()[-1] + " " + self.id2unigram[chosen]]
            else:
                current_idx = self.ngram2id[start]
                
        return '  '.join(text)

In [268]:
news_generator = TextGenerator(news.tokens_by_sentence, 3) # to be modified to work on any ngram >= 2
news_generator.generate_ngrams()
news_generator.build_matrix()

In [295]:
print(news_generator.generate_text().replace('<end>', '\n'))

об  этом  сегодня  пишут  все  газеты  мира  сказал  лидер  яблока  
  сша  планирующие  развернуть  новую  национальную  систему  противоракетной  обороны  хотят  добиться  изменений  в  договоре  по  про  поскольку  настоящая  версия  договора  не  позволяет  развертывание  нпро  
  ожидается  что  в  пятницу  генпрокуратура  рф  отстранило  трех  ведущих  следователей  от  расследования  других  дел  о  коррупции  вооруженные  отряды  из  восточнотиморской  столицы  дили  официальный  представитель  генерального  секретаря  оон  фред  экхардзаявил  в  минувший  вторник  в  американской  печати  появились  выдержки  из  служебного  письма  адресованного  председателем  совета  директоров  мвф  доклад  о  результатах  своих  переговоров  с  возглавляемым  сша  командованием  сил  оон  провозгласила  свою  собственую  морскую  границу  60-70  километрами  южнее  существующей


In [265]:
print(news_generator.generate_text().replace('<end>', '\n'))

как  заявили  интерфаксу  в  четверг  подал  в  останкинский  межмуниципальный  суд  иск  к  орт  о  защите  чести  и  достоинства  
  в  рамках  мандата  позволяющего  миротворцам  применять  силу  для  самозащиты  
  входе  перестрелки  с  правительственными  войсками  один  боевиксорвался  в  пропасть  и  предположительно  погиб  
  в  вышедшей  в  эфир  как  сообщила  пресс-службе  штаба  боос  выступил  с  протестом  против  вольной  трактовки  своего  интервью  расценив  ее  как  намеренное  искажение  его  слов  и  мыслей  
  главным  образом  считает  источник  это  связано  с  тем  возможность  переговоров  и  исламистами  вовсе  не  означает  сказал  махтигаджиев  что  все  миротворцы  следуют  этому  призыву  констатирует  мид  рф  распространил  сообщение  в  котором  говорится


In [266]:
print(news_generator.generate_text().replace('<end>', '\n'))

в  итоге  фас  мо  удовлетворил  кассационные  жалобы  инкомбанка  и  его  секретные  коды  
  об  этом  интерфаксу  сообщили  что  дневной  заработок  хлебного  спекулянта  достигает  минимум  ста  рублей  но  пресечь  этот  бизнес  нельзя  поскольку  перепродажа  если  она  не  связана  с  проведением  осенней  сессии  руководящих  органов  мвф  и  российским  правительством  и  за  его  голову  фбр  обещаловыплатить  5  млн  рублей  
  содержание  алкоголя  в  крови  водителя  лимузина  генри  пола  в  три  раза  превосходило  норму  1,8  грамма  на  литр  
  как  сообщили  интерфаксу  в  четверг  вечером  представители  генеральной  прокуратуры  россии  приступили  к  обыскам  на  даче  и  в  еще  одной  большой  стране  по  выражению  александра  лившица


# Task 2
Задание 2. (5 баллов) Напишите функцию оценивания нграммов на основе PMI. Используйте эту функцию вместо дефолтной в gensim.models.Phrases Обучите два последовательных модели Phrases с такой функцией и проанализируйте результаты, получаемые после преобразования текстов двумя Phrases.

Пояснения: Формулу PMI можно посмотреть вот тут https://en.wikipedia.org/wiki/Pointwise_mutual_information , также там описано как вывести вероятности из количества вхождений слова1, слова2, нграмма и размера корпуса. Чтобы функцию можно было поставить в аргумент scoring в Phrases у нее должны быть вот такие аргументы - scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count) Подберите параметр threshold под эту функцию. Чтобы проверить, что она вообще работает поставьте какое-то совсем маленькое число, чтобы порог проходили все нграммы и потом постепенно его повышайте. В тетрадке есть пример обучения нескольких Phrases последовательно, воспользуйтесь им.

In [21]:
import numpy as np
import gensim

def scorer_pmi(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
    """ 
        a function to evaluate ngrams to be used in gensim.Phraser
        pmi formula is based on table retrieved from the following link
        https://en.wikipedia.org/wiki/Pointwise_mutual_information
    """
    try:
        score = np.log(
            (bigram_count / worda_count) / 
            (wordb_count / corpus_word_count))
    
    except ZeroDivisionError:
        return 0
    
    return score

In [22]:
scorer_pmi(1938, 1311, 1159, 1000, 1000, 50000952)

10.034908170336502

Дефолтная функция на низком пороге работает более устойчиво, не захватывая лишний мусор, созданная функция захватывает почти любые последовательсности

На более высоком пороге, дефолтная функция также лучше справляется, позволяя устанавливать цепочки длиннее: 'со_ссылкой_на', 'в_том_числе'

In [23]:
corpus = news.tokens_by_sentence[0:50000]
# собираем статистики
ph = gensim.models.Phrases(corpus, threshold = 15, scoring='default')
# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)
# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)
for i in range(5):
    print(p2[p[corpus[50+i]]])

['однако', 'уточненная', 'оценка', 'числа', 'пострадавших', 'в_результате', 'этого', 'взрыва', 'может', 'достигнуть', 'ста', 'человек']
['агентство', 'итар-тасс', 'в', 'сообщении', 'от', '21.15', 'со_ссылкой_на', 'источники', 'в', 'гувд', 'москвы', 'говорит', 'только', 'о', '30', 'раненых', 'в_том_числе', 'о', 'двух', 'пострадавших', 'в', 'тяжелом', 'состоянии']
['однако', 'число', 'пострадавших', 'в_результате', 'этого', 'взрыва', 'может', 'составить', 'до', 'ста', 'человек']
['по_данным', 'риа_новости', 'боткинская', 'больница', 'институт', 'им']
['склифосовского', '1-ая', 'градская', '36-ая', 'и', '64-ая', 'горбольница', 'работают', 'только', 'на', 'прием', 'пострадавших']


In [24]:
corpus = news.tokens_by_sentence[0:50000]
# собираем статистики
ph = gensim.models.Phrases(corpus, threshold = 15, scoring=scorer_pmi)
# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)
# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)
for i in range(5):
    print(p2[p[corpus[50+i]]])

['однако', 'уточненная', 'оценка', 'числа', 'пострадавших', 'в_результате', 'этого', 'взрыва', 'может', 'достигнуть', 'ста', 'человек']
['агентство', 'итар-тасс', 'в', 'сообщении', 'от', '21.15', 'со_ссылкой', 'на', 'источники', 'в', 'гувд', 'москвы', 'говорит', 'только', 'о', '30', 'раненых', 'в_том', 'числе', 'о', 'двух', 'пострадавших', 'в', 'тяжелом', 'состоянии']
['однако', 'число', 'пострадавших', 'в_результате', 'этого', 'взрыва', 'может', 'составить', 'до', 'ста', 'человек']
['по_данным', 'риа_новости', 'боткинская', 'больница', 'институт', 'им']
['склифосовского', '1-ая', 'градская', '36-ая', 'и', '64-ая', 'горбольница', 'работают', 'только', 'на', 'прием', 'пострадавших']


In [96]:
p2.scoring

<function gensim.models.phrases.original_scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count)>

In [102]:
series = 'black white blue'.split()
" ".join(series[:-1])

'black white'

In [104]:
series[-1]

'blue'

In [227]:
" ".join(['<start>']*4)

'<start> <start> <start> <start>'